In [14]:
import numpy as np
import scipy.stats as stats
from scipy.special import betainc


In [15]:

def Fischer_test(sample1, sample2, alternative="two-sided"):
    """Проверка гипотезы о равенстве двух генеральных дисперсий"""
    n = len(sample1)
    m = len(sample2)
    f = np.var(sample1, ddof=1)/np.var(sample2, ddof=1)
    match(alternative):
        case "less":
            return stats.f.cdf(f, n-1, m-1)
        case "greater":
            return 1 - stats.f.cdf(f, n-1, m-1)
        case "two-sided":
            return 2*min(stats.f.cdf(f, n-1, m-1), 1 - stats.f.cdf(f, n-1, m-1))



In [16]:
def Welch_t_test(sample1, sample2, alternative='two-sided'):
    """Реализация t-теста Уэлча для двух независимых выборок."""
    n1 = len(sample1)
    n2 = len(sample2)
    mean1 = np.mean(sample1)
    mean2 = np.mean(sample2)
    var1 = np.var(sample1, ddof=1)  # Исправленная дисперсия
    var2 = np.var(sample2, ddof=1)

    # t-статистика
    t = (mean1 - mean2) / np.sqrt(var1/n1 + var2/n2)
    
    # Степени свободы (Саттертуэйт)
    df = (var1/n1 + var2/n2)**2 / ((var1/n1)**2/(n1-1) + (var2/n2)**2/(n2-1))
    
    # p-значение (используем CDF t-распределения)
    def t_cdf(x, df):
        # Реализация CDF t-распределения через бета-функцию
        x_sq = x**2
        p = betainc(0.5*df, 0.5, df/(df + x_sq))
        return 0.5 * (1 + np.sign(x) * (1 - p))
    
    p_two_sided = 2 * (1 - t_cdf(np.abs(t), df))
    
    if alternative == 'two-sided':
        p_value = p_two_sided
    elif alternative == 'greater':
        p_value = 1 - t_cdf(t, df)
    elif alternative == 'less':
        p_value = t_cdf(t, df)
    else:
        raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")
    
    return p_value

In [17]:

def Z_test(sample1, sample2, std1, std2, alternative="two-sided"):
    n = len(sample1)
    m = len(sample2)
    z = (np.mean(sample1) - np.mean(sample2))/(std1**2/n + std2**2/m)**.5

    match(alternative):
        case "less":
            return stats.norm.cdf(z)
        case "greater":
            return 1 - stats.norm.cdf(z)
        case "two-sided":
            return stats.norm.cdf(-np.abs(z))


In [18]:
def T_test(sample1, sample2, alternative="two-sided"):
    n = len(sample1)
    m = len(sample2)
    std1 = np.std(sample1, ddof=1) # Несмещенная оценка
    std2 = np.std(sample2, ddof=1) # Несмещенная оценка

    t = ((np.mean(sample1) - np.mean(sample2))/((n-1)*std1**2 + (m-1)*std2**2)**.5) * ((n*m*(n+m-2))/(n+m))**.5
    match(alternative):
        case "less":
            return stats.t.cdf(t, n+m-2)
        case "greater":
            return 1 - stats.t.cdf(t, n+m-2)
        case "two-sided":
            # Ошибка: возвращает половину p-value
            return stats.t.cdf(-np.abs(t), n+m-2)



In [19]:

def dependent_t_test(sample1, sample2):
    """Проверка гипотезы о равенстве двух генеральных средних, когда выборки зависимы"""
    if len(sample1) != len(sample2): raise BaseException("Для теста с зависимыми выборками, размер выборок должен совпадать")
    n = len(sample1)
    D = np.array(sample1) - np.array(sample2)
    # Ошибка: используется смещенная оценка std (np.std по умолчанию ddof=0)
    # и возвращается половина p-value для двусторонней гипотезы
    t = (np.mean(D)*n**.5)/np.std(D) # Должно быть np.std(D, ddof=1)
    return stats.t.cdf(-np.abs(t), n-1) # Возвращает половину p-value



In [ ]:
def universal_mean_test(sample1, sample2, alternative="two-sided", std1=None, std2=None, isdependent=False):
    if isdependent:
        # t-test для зависимых выборок
        return dependent_t_test(sample1, sample2)
    elif std1 and std2:
        # z-test для известных дисперсий (независимые выборки)
        return Z_test(sample1, sample2, std1, std2, alternative)
    else:
        if Fischer_test(sample1, sample2) > 0.05:
            return T_test(sample1, sample2, alternative)
        else:
            return Welch_t_test(sample1, sample2, alternative)

In [21]:
def universal_mean_test_scipy(sample1, sample2, alternative="two-sided", std1=None, std2=None, isdependent=False, alpha=0.05):
    """Сравнение с использованием scipy.stats"""
    p_value = None
    if isdependent:
        p_value = stats.ttest_rel(sample1, sample2, alternative=alternative)[1]

    elif std1 and std2:
        # z-test для известных дисперсий (независимые выборки)
        n = len(sample1)
        m = len(sample2)
        z_statistic = (np.mean(sample1) - np.mean(sample2)) / (std1**2/n + std2**2/m)**.5
        # Корректный расчет p-value
        if alternative == 'less':
            p_value = stats.norm.cdf(z_statistic)
        elif alternative == 'greater':
            p_value = 1 - stats.norm.cdf(z_statistic)
        elif alternative == 'two-sided':
            p_value = 2 * stats.norm.cdf(-np.abs(z_statistic)) # Правильная формула
    else:
        # t-test для неизвестных дисперсий (независимые выборки)
        # Проверка равенства дисперсий (как в universal_mean_test)
        fischer_p_value = Fischer_test(sample1, sample2)
        if fischer_p_value < alpha:
            p_value = stats.ttest_ind(sample1, sample2, alternative=alternative, equal_var=False)[1]
        else:
            p_value = stats.ttest_ind(sample1, sample2, alternative=alternative, equal_var=True)[1]
    
    return p_value

# Решение Задач

In [22]:

# Задача 1
sample1_z1 = [130] * 30
sample2_z1 = [125] * 40
std1_z1 = 60**(1/2)
std2_z1 = 80**(1/2)

result_z1 = universal_mean_test(sample1_z1, sample2_z1, alternative="two-sided", std1=std1_z1, std2=std2_z1)
result_scipy_z1 = universal_mean_test_scipy(sample1_z1, sample2_z1, alternative="two-sided", std1=std1_z1, std2=std2_z1)
print(f"Результат теста (мой) : {result_z1}")
print(f"Результат теста (scipy): {result_scipy_z1}")
print(f"Вывод: p = {result_scipy_z1:.4f}")


Результат теста (мой) : 0.006209665325776132
Результат теста (scipy): 0.012419330651552265
Вывод: p = 0.0124


In [ ]:

# Задача 2
sample1_z2 = [3.4]*2+[3.5]*3+[3.7]*4+[3.9]
sample2_z2 = [3.2]*2+[3.4]*2+[3.6]*8

result_z2 = universal_mean_test(sample1_z2, sample2_z2, alternative="two-sided")
result_scipy_z2 = universal_mean_test_scipy(sample1_z2, sample2_z2, alternative="two-sided", alpha=0.05) # Используем alpha=0.05 для проверки дисперсий в scipy-функции
print(f"Результат теста (мой) : {result_z2}")
print(f"Результат теста (scipy): {result_scipy_z2}")
print(f"Вывод: p = {result_scipy_z2:.5f}")


Результат теста (мой) : 0.0814992575109142
Результат теста (scipy): 0.16299851502182822
Вывод: p = 0.16300


In [24]:

# Задача 3
sample1_z3 = [2, 3, 5, 6, 8, 10]
sample2_z3 = [10, 3, 6, 1, 7, 4]
result_z3 = universal_mean_test(sample1_z3, sample2_z3, alternative="two-sided", isdependent=True)
result_scipy_z3 = universal_mean_test_scipy(sample1_z3, sample2_z3, alternative="two-sided", isdependent=True)

print(f"Результат теста (мой) : {result_z3}")
print(f"Результат теста (scipy): {result_scipy_z3}")
print(f"Вывод: p = {result_scipy_z3:.3f}")

Результат теста (мой) : 0.39977728803854506
Результат теста (scipy): 0.8165892159697636
Вывод: p = 0.817
